<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/best_practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Best practices
In this notebook we'll see how to improve a model's performance by tuning its architecture-level hyperparameters such as:

* the number of layers
* the number of units, and filters of a layer
* the activation function of a layer
* the amount of dropout
* batch normalization layers
* the optimizer and its learning rate

The tuning is a search in the hyperparameters space that is much better to do automatically and systematically rather than manually. Keras provides a tool, [KerasTuner](https://keras.io/keras_tuner/getting_started/), to perform the search of the optimal hyperparameters. The tool allows us to set a range of values for each hyperparameter to search for instead of only one. In order to use the tool we have to define a function to build our model and to pass the values of the parameters set by the tool. We start by downloading the tool.

In [1]:
import tensorflow as tf
import keras
from keras import layers
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.15.0


## Tensor Processing Unit
In this notebook we will use a [TPU](https://colab.research.google.com/notebooks/tpu.ipynb). This is not strictly necessary but it will be a test of the Google processing unit to compare with standard CPU or GPU.

In [2]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print(f'Running on a TPU w/{tpu.num_accelerators()["TPU"]} cores')
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on a TPU w/8 cores


In [3]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


## The model-building function
We define a function that accept a variable that will be used by the Keras Tuner to pass the hyperparameters to build the model. The model will be created, compiled and returned to be fit with the data. A a first example we define a function to build a model with two fully connected layers and

* a variable number of units in the first layer, between 16 and 64, with a step of 16, that is 16, 32, 48, and 64
* two different optimizers: rmsprop and adam

In [4]:
def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])

    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

    return model

We can achieve the same result in a more modular way by subclassing the HyperModel class and overriding the _build()_ function

In [5]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])

        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])

        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

        return model

hypermodel = SimpleMLP(num_classes=10)

## The tuner
The hyperparameters search space for our model has 4 * 2 = 8 possible states. The Keras Tuner will build, fit, and assess the performances of the model trying all the possible values of the hyperparameters automatically, and it will finally store the best model in a directory. In our example the tuner will assess the validation accuracy of the models. The tuner can use different algorithms for its search in the hyperparameter space: random search, grid search, Bayesian search, and others.   

In [6]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=10, #100
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


## Looking for the best model using the MNIST dataset
We look for the best hyperparameters settings for a model that will be used in a classification task with the MNIST dataset. The MNIST dataset is loaded from a URL and the data is kept in the memory available to the VM that execute the notebook. Using the TPU is much faster than a GPU (e.g.: a T4 GPU). The batch size for a TPU should be large in order to use it effectively, e.g. 1024 or more.

In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]

num_epochs = 20

tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=num_epochs, # 100
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 10 Complete [00h 00m 43s]
val_accuracy: 0.9751499891281128

Best val_accuracy So Far: 0.9762000143527985
Total elapsed time: 00h 06m 53s


## Best hyperparameters configuration
After the hyperparameters search is complete we can use the best model, that with the highest rank. We can retrain it using a higher number of epochs and early stopping to stop the retraining when it starts to overfit. We can select the first 4 set of hyperparamters   

In [9]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

The best set, index 0, is the set of hyperparameters with the highest accuracy. We can see its hyperparameters

In [10]:
best_hp = best_hps[0]
best_hp.values

{'units': 64, 'optimizer': 'rmsprop'}

## Best model retraining
We can retrain one or more of the best models using their hypeparameters and a higher number of epochs with early stopping to stop the retraining when it starts to overfit. We can select the first 4 sets of hyperparamters

In [11]:
def get_best_epoch(hp):
    model = build_model(hp)

    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=num_epochs, #100
        batch_size=128,
        callbacks=callbacks)

    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [13]:
best_epoch = get_best_epoch(best_hp)
best_epoch

Epoch 1/20
391/391 [==============================] - 2s 3ms/step - loss: 0.4214 - accuracy: 0.8867 - val_loss: 0.2299 - val_accuracy: 0.9340
Epoch 2/20
391/391 [==============================] - 1s 3ms/step - loss: 0.2157 - accuracy: 0.9383 - val_loss: 0.1801 - val_accuracy: 0.9491
Epoch 3/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1654 - accuracy: 0.9518 - val_loss: 0.1554 - val_accuracy: 0.9558
Epoch 4/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1338 - accuracy: 0.9610 - val_loss: 0.1326 - val_accuracy: 0.9613
Epoch 5/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1135 - accuracy: 0.9667 - val_loss: 0.1158 - val_accuracy: 0.9648
Epoch 6/20
391/391 [==============================] - 1s 3ms/step - loss: 0.0984 - accuracy: 0.9716 - val_loss: 0.1124 - val_accuracy: 0.9679
Epoch 7/20
391/391 [==============================] - 1s 3ms/step - loss: 0.0862 - accuracy: 0.9748 - val_loss: 0.1056 - val_accuracy: 0.9674
Epoch 

19

We use more epochs for the retraining of the best models and we use the full dataset without the validation set

In [14]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

We retrain the best models

In [15]:
best_models_retrained = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models_retrained.append(model)

Epoch 1/20
391/391 [==============================] - 2s 3ms/step - loss: 0.4276 - accuracy: 0.8857 - val_loss: 0.2387 - val_accuracy: 0.9328
Epoch 2/20
391/391 [==============================] - 1s 3ms/step - loss: 0.2226 - accuracy: 0.9368 - val_loss: 0.1774 - val_accuracy: 0.9507
Epoch 3/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1732 - accuracy: 0.9503 - val_loss: 0.1507 - val_accuracy: 0.9594
Epoch 4/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1427 - accuracy: 0.9592 - val_loss: 0.1335 - val_accuracy: 0.9613
Epoch 5/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1208 - accuracy: 0.9654 - val_loss: 0.1201 - val_accuracy: 0.9666
Epoch 6/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1044 - accuracy: 0.9702 - val_loss: 0.1149 - val_accuracy: 0.9660
Epoch 7/20
391/391 [==============================] - 1s 3ms/step - loss: 0.0915 - accuracy: 0.9741 - val_loss: 0.1071 - val_accuracy: 0.9683
Epoch 

In [16]:
best_model_retrained = best_models_retrained[0]
best_model_retrained.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                50240     
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                                 
Total params: 50890 (198.79 KB)
Trainable params: 50890 (198.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


If we do not want to retrain the best models we can simply get them from the tuner

In [19]:
best_models = tuner.get_best_models(top_n)
len(best_models)

4